In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score


In [ ]:
# Cell 2: Transforms
weights = ResNet50_Weights.DEFAULT
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    weights.transforms()
    ])

In [ ]:
# Cell 3: Data Loaders
trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/CIFAR-10-images-master/CIFAR-10-images-master/train', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/CIFAR-10-images-master/CIFAR-10-images-master/test', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)
classes = trainset.classes

In [ ]:
# Cell 4: Model Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet50(weights=weights)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 156MB/s]


In [ ]:
# Cell 5: Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Cell 6: Training Loop
for epoch in range(2):  # Adjust as needed
    running_loss = 0.0
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

print("Finished Training")


[1,   100] loss: 0.926
[1,   200] loss: 0.563
[1,   300] loss: 0.525
[1,   400] loss: 0.456
[1,   500] loss: 0.415
[1,   600] loss: 0.397
[1,   700] loss: 0.384
[2,   100] loss: 0.288
[2,   200] loss: 0.271
[2,   300] loss: 0.284
[2,   400] loss: 0.293
[2,   500] loss: 0.312
[2,   600] loss: 0.277
[2,   700] loss: 0.288
Finished Training


In [ ]:
# Cell 7: Evaluation with Accuracy and F1 Score
correct = 0
total = 0
all_labels = []
all_preds = []

model.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Accuracy on the 10000 test images: {accuracy:.2f}%")
print(f"Weighted F1 Score: {f1:.4f}")

Accuracy on the 10000 test images: 88.70%
Weighted F1 Score: 0.8874
